In [1]:
%pip install us
%pip install easymoney

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests, csv 
import us
import pandas as pd
import time
from easymoney.money import EasyPeasy
import json


/Users/gabrielmelmed/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [9]:
api_key = input("API Key")

### Constructing County Calls

In [12]:
dataset = 'acs/acs1'
series = "B21004_001E"
state = 17

In [13]:
county_responses = []
for year in list(range(2005, 2020))+[2021]:
    data_url = f'https://api.census.gov/data/{year}/{dataset}?get=NAME,{series}&for=county:*&in=state:{state}&key={api_key}'
    print(f'{year}: {data_url}')
    df = pd.read_json(data_url)
    df.columns = df.iloc[0]
    df = df[1:]
    df['Year'] = year
    time.sleep(0.1)
    county_responses.append(df)

2005: https://api.census.gov/data/2005/acs/acs1?get=NAME,B21004_001E&for=county:*&in=state:17&key=9330018ae4a2bd20da22ced1281145d9405dc349
2006: https://api.census.gov/data/2006/acs/acs1?get=NAME,B21004_001E&for=county:*&in=state:17&key=9330018ae4a2bd20da22ced1281145d9405dc349
2007: https://api.census.gov/data/2007/acs/acs1?get=NAME,B21004_001E&for=county:*&in=state:17&key=9330018ae4a2bd20da22ced1281145d9405dc349
2008: https://api.census.gov/data/2008/acs/acs1?get=NAME,B21004_001E&for=county:*&in=state:17&key=9330018ae4a2bd20da22ced1281145d9405dc349
2009: https://api.census.gov/data/2009/acs/acs1?get=NAME,B21004_001E&for=county:*&in=state:17&key=9330018ae4a2bd20da22ced1281145d9405dc349
2010: https://api.census.gov/data/2010/acs/acs1?get=NAME,B21004_001E&for=county:*&in=state:17&key=9330018ae4a2bd20da22ced1281145d9405dc349
2011: https://api.census.gov/data/2011/acs/acs1?get=NAME,B21004_001E&for=county:*&in=state:17&key=9330018ae4a2bd20da22ced1281145d9405dc349
2012: https://api.census.go

merging dataframes and filtering for Chicago + Collar Counties

In [14]:
#merge all dataframes in responses into one dataframe
df = pd.concat(county_responses)

counties = [31, #Cook
    43, #Dupage
    89, #Kane
    97, #Lake
    111, #McHenry
    197 #Will
]

#filter to only the counties we want
df.county = df.county.astype(int)
df = df[df['county'].isin(counties)]

# keep only name column before comma
df['NAME'] = df['NAME'].str.split(',').str[0]

#rename columns
df = df.rename(columns={'B21004_001E': 'Income'})
county_df = df.copy()

In [ ]:
counties

### Getting Census Tract Data

In [15]:
dataset = 'acs/acs5' # American Community Survey 5-year data
series = "B21004_001E" # Median Income
state = 17 # Illinois

responses = [] 
for year in list(range(2010, 2020))+[2021]: # Loop through years, there's no data for 2020
    data_url = f'https://api.census.gov/data/{year}/{dataset}?get=NAME,{series}&for=tract:*&in=state:{state}&in=county:*&key={api_key}'
    print(f'{year}: {data_url}')
    df = pd.read_json(data_url) # Read data from API into a pandas dataframe
    df.columns = df.iloc[0] # Set first row as column names
    df = df[1:] # Remove first row
    df["Year"] = year
    time.sleep(1) # Sleep for 1 second to avoid hitting the API too fast
    responses.append(df)

2010: https://api.census.gov/data/2010/acs/acs5?get=NAME,B21004_001E&for=tract:*&in=state:17&in=county:*&key=9330018ae4a2bd20da22ced1281145d9405dc349
2011: https://api.census.gov/data/2011/acs/acs5?get=NAME,B21004_001E&for=tract:*&in=state:17&in=county:*&key=9330018ae4a2bd20da22ced1281145d9405dc349
2012: https://api.census.gov/data/2012/acs/acs5?get=NAME,B21004_001E&for=tract:*&in=state:17&in=county:*&key=9330018ae4a2bd20da22ced1281145d9405dc349
2013: https://api.census.gov/data/2013/acs/acs5?get=NAME,B21004_001E&for=tract:*&in=state:17&in=county:*&key=9330018ae4a2bd20da22ced1281145d9405dc349
2014: https://api.census.gov/data/2014/acs/acs5?get=NAME,B21004_001E&for=tract:*&in=state:17&in=county:*&key=9330018ae4a2bd20da22ced1281145d9405dc349
2015: https://api.census.gov/data/2015/acs/acs5?get=NAME,B21004_001E&for=tract:*&in=state:17&in=county:*&key=9330018ae4a2bd20da22ced1281145d9405dc349
2016: https://api.census.gov/data/2016/acs/acs5?get=NAME,B21004_001E&for=tract:*&in=state:17&in=coun

In [189]:
#merge all dataframes in responses into one dataframe
df = pd.concat(responses)

#filter to only the counties we want
counties = [31, 43, 89, 97, 111, 197] # Cook, Dupage, Kane, Lake, McHenry, Will; Chicago's collar counties
df.county = df.county.astype(int)
df = df[df['county'].isin(counties)]

# keep only name column before comma
df['County_Name'] = df['NAME'].str.split(',').str[1]
df['NAME'] = df['NAME'].str.split(',').str[0]
df = df.rename(columns={'B01001_001E': 'Income'})

In [17]:
pd.concat(responses)

,NAME,B21004_001E,state,county,tract,Year
1,"Census Tract 2516, Cook County, Illinois",13922,17,031,251600,2010
2,"Census Tract 2517, Cook County, Illinois",16577,17,031,251700,2010
3,"Census Tract 2518, Cook County, Illinois",16173,17,031,251800,2010
4,"Census Tract 2519, Cook County, Illinois",12949,17,031,251900,2010
5,"Census Tract 2520, Cook County, Illinois",18158,17,031,252000,2010
...,...,...,...,...,...,...
3261,"Census Tract 305.01, Woodford County, Illinois",48365,17,203,030501,2021
3262,"Census Tract 305.02, Woodford County, Illinois",32975,17,203,030502,2021
3263,"Census Tract 306.01, Woodford County, Illinois",28937,17,203,030601,2021
3264,"Census Tract 306.02, Woodford County, Illinois",44194,17,203,030602,2021


In [190]:
df

,NAME,B21004_001E,state,county,tract,Year,County_Name
1,Census Tract 2516,13922,17,31,251600,2010,Cook County
2,Census Tract 2517,16577,17,31,251700,2010,Cook County
3,Census Tract 2518,16173,17,31,251800,2010,Cook County
4,Census Tract 2519,12949,17,31,251900,2010,Cook County
5,Census Tract 2520,18158,17,31,252000,2010,Cook County
...,...,...,...,...,...,...,...
3154,Census Tract 8840.06,39329,17,197,884006,2021,Will County
3155,Census Tract 8841.01,27486,17,197,884101,2021,Will County
3156,Census Tract 8841.03,42837,17,197,884103,2021,Will County
3157,Census Tract 9800,-666666666,17,197,980000,2021,Will County


In [191]:
df.to_excel('Chicago_subdivisions.xlsx')

### Adjusting For Real Incomes

In [192]:
ep = EasyPeasy(fuzzy_threshold=True)

In [195]:
county_df

,NAME,Income,state,county,Year
3,Cook County,27117,17,31,2005
5,DuPage County,36315,17,43,2005
6,Kane County,29900,17,89,2005
9,Lake County,33733,17,97,2005
11,McHenry County,35412,17,111,2005
...,...,...,...,...,...
9,Cook County,39209,17,31,2021
11,DuPage County,48358,17,43,2021
12,Kane County,40830,17,89,2021
15,McHenry County,41799,17,111,2021


In [201]:
# rename B21004_001E to Income
county_df = county_df.rename(columns={'B21004_001E': 'Income'})
# Drop all rows with NaN values
county_df = county_df.dropna()
# Drop all negative values
county_df = county_df[county_df['Income'] >= 0]
county_df['Income'] = county_df['Income'].astype(int)
county_df['Real Income'] = county_df.apply(lambda x: ep.normalize(amount=x.Income, region="United States", from_year=x.Year, to_year="latest", base_currency="USD"), axis=1)

In [202]:
# do the same to df
df = df.rename(columns={'B21004_001E': 'Income'})
df = df.dropna()
# Drop all rows with negative incomes
df = df[df['Income'] > 0]
df['Income'] = df['Income'].astype(int)
df['Real Income'] = df.apply(lambda x: ep.normalize(amount=x.Income, region="United States", from_year=x.Year, to_year="latest", base_currency="USD"), axis=1)

In [207]:
df['Real Income'].describe()

count     21267.000000
mean      39217.162832
std       16486.681981
min        2860.370000
25%       26946.055000
50%       36140.190000
75%       48439.605000
max      187560.000000
Name: Real Income, dtype: float64

## Exporting Data

In [203]:
county_df.to_json('Chicago_counties_incomes.json', orient='records')
df.to_json('Chicago_wards_incomes.json', orient='records')

### Filtering the GeoJson

In [170]:
# parse Census_tracts.topojson
with open('/Users/finn/Documents/GitHub/data/e4e3/Notebooks/cb_2018_17_tract_500k.json') as f:
    geojson = json.load(f)

In [173]:
geojson['features'] = [feature for feature in geojson['features'] if int(feature['properties']['COUNTYFP']) in counties]

In [174]:
# write geojson to 'chicago_tracts.json'
with open('chicago_tracts.json', 'w') as f:
    json.dump(geojson, f)

In [171]:
geojson.keys()

dict_keys(['type', 'features'])

In [176]:
df.sort_values(by=['Year', 'Income'], ascending=False)

,NAME,Income,state,county,tract,Year,County_Name,Real Income
2090,Census Tract 8609.06,11346,17,97,860906,2021,Lake County,11346.0
292,Census Tract 818,11295,17,31,081800,2021,Cook County,11295.0
1473,Census Tract 8435,10528,17,31,843500,2021,Cook County,10528.0
1436,Census Tract 8390,10155,17,31,839000,2021,Cook County,10155.0
2113,Census Tract 8615.04,9458,17,97,861504,2021,Lake County,9458.0
...,...,...,...,...,...,...,...,...
1068,Census Tract 5613,0,17,31,561300,2009,Cook County,0.0
1083,Census Tract 5810,0,17,31,581000,2009,Cook County,0.0
1144,Census Tract 6402,0,17,31,640200,2009,Cook County,0.0
1261,Census Tract 7609,0,17,31,760900,2009,Cook County,0.0
